<h4><b>Importing necessary libraries and connecting to the database</b></h4>

In [30]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Connect to SQLite database
conn = sqlite3.connect("../ecommerce.db")

<h4><b>Loading and joining tables as dataset</b></h4>

In [31]:
fact_sales = pd.read_sql("SELECT * FROM fact_sales", conn)
dim_date = pd.read_sql("SELECT date_key, date FROM dim_date", conn)

fact_sales = fact_sales.merge(dim_date, on="date_key", how="left")
fact_sales["date"] = pd.to_datetime(fact_sales["date"])

<h4><b>Defining Analysis Date (For Recency Calculations)</b></h4>

In [32]:
analysis_date = fact_sales["date"].max()
analysis_date

Timestamp('2011-12-09 00:00:00')

<h4><b>Computing Recency</b></h4>

In [33]:
recency = fact_sales.groupby("customer_id")["date"].max().reset_index()
recency["recency_days"] = (analysis_date - recency["date"]).dt.days
recency.head()

,customer_id,date,recency_days
0,12346,2011-01-18,325
1,12347,2011-12-07,2
2,12348,2011-09-25,75
3,12349,2011-11-21,18
4,12350,2011-02-02,310


<h4><b>Computing Frequency</b></h4>

In [34]:
frequency= fact_sales.groupby("customer_id").size().reset_index(name="frequency")
frequency.head()

,customer_id,frequency
0,12346,1
1,12347,182
2,12348,31
3,12349,73
4,12350,17


<h4><b>Computing Monetary value and log transforming it</b></h4>

In [35]:
monetary=fact_sales.groupby("customer_id",as_index=False)["revenue"].sum().rename(columns={"revenue":"monetary"})
monetary["monetary_log"] = np.log1p(monetary["monetary"]) #we are using log1p(log(1+x)) so that the code doesnt break in the event of unexpected 0
monetary.describe()

,customer_id,monetary,monetary_log
count,4338.000000,4338.000000,4338.000000
mean,15300.408022,2054.266460,6.593627
std,1721.808492,8989.230441,1.257578
min,12346.000000,3.750000,1.558145
25%,13813.250000,307.415000,5.731446
50%,15299.500000,674.485000,6.515431
75%,16778.750000,1661.740000,7.416222
max,18287.000000,280206.020000,12.543284


<h4><b>Merging Recency , Frequency , Monetary(R,F,M)</b><h4>

In [36]:
rfm = monetary.merge(recency, on="customer_id",how="inner").merge(frequency,on="customer_id",how="inner")
rfm.rename(columns={"date":"last_purchase_date"},inplace=True)
rfm.head()

,customer_id,monetary,monetary_log,last_purchase_date,recency_days,frequency
0,12346,77183.60,11.253955,2011-01-18,325,1
1,12347,4310.00,8.368925,2011-12-07,2,182
2,12348,1797.24,7.494564,2011-09-25,75,31
3,12349,1757.55,7.472245,2011-11-21,18,73
4,12350,334.40,5.815324,2011-02-02,310,17


<h4><b>Quantile RFM Scoring (5 point based)</b><h4>

In [37]:
# Recency Score
rfm["r_score"] = pd.qcut(rfm["recency_days"],5,labels=[5,4,3,2,1]) #in case of recency fewer days is better, hence opposite direction of labelling
# Frequency Score
rfm["f_score"]=pd.qcut(rfm["frequency"],5,labels=[1,2,3,4,5])
# Monetary Score
rfm["m_score"]=pd.qcut(rfm["monetary_log"],5,labels=[1,2,3,4,5])
rfm[["r_score","f_score","m_score"]].astype(int).describe()

,r_score,f_score,m_score
count,4338.000000,4338.000000,4338.00000
mean,3.006455,2.978792,3.00000
std,1.413220,1.429300,1.41454
min,1.000000,1.000000,1.00000
25%,2.000000,2.000000,2.00000
50%,3.000000,3.000000,3.00000
75%,4.000000,4.000000,4.00000
max,5.000000,5.000000,5.00000


<h4><b>Building RFM code </h4><b>

In [38]:
rfm["rfm_code"]=rfm["r_score"].astype("str")+rfm["f_score"].astype("str") + rfm["m_score"].astype("str")
rfm.head()

,customer_id,monetary,monetary_log,last_purchase_date,recency_days,frequency,r_score,f_score,m_score,rfm_code
0,12346,77183.60,11.253955,2011-01-18,325,1,1,1,5,115
1,12347,4310.00,8.368925,2011-12-07,2,182,5,5,5,555
2,12348,1797.24,7.494564,2011-09-25,75,31,2,3,4,234
3,12349,1757.55,7.472245,2011-11-21,18,73,4,4,4,444
4,12350,334.40,5.815324,2011-02-02,310,17,1,2,2,122


<h4><b>Implementing Segmentation Logic</b></h4>

In [ ]:
def rfm_segmentation(row):
    r = int(row["r_score"])
    f = int(row["f_score"])
    m = int(row["m_score"])
    
 # Activeness: recent OR (moderately recent AND frequent)
    active = (r >= 4) or (r >= 3 and f >= 4)

    # Value: high frequency OR high monetary
    high_value = (m >= 4)

    if active and high_value:
        return "Active High-Value"
    elif active and not high_value:
        return "Active Low-Value"
    elif not active and high_value:
        return "Inactive High-Value"
    else:
        return "Inactive Low-Value"
    

rfm["segments"] = rfm.apply(rfm_segmentation, axis=1)
rfm.head()


,customer_id,monetary,monetary_log,last_purchase_date,recency_days,frequency,r_score,f_score,m_score,rfm_code,segments
0,12346,77183.60,11.253955,2011-01-18,325,1,1,1,5,115,Inactive High-Value
1,12347,4310.00,8.368925,2011-12-07,2,182,5,5,5,555,Active High-Value
2,12348,1797.24,7.494564,2011-09-25,75,31,2,3,4,234,Inactive High-Value
3,12349,1757.55,7.472245,2011-11-21,18,73,4,4,4,444,Active High-Value
4,12350,334.40,5.815324,2011-02-02,310,17,1,2,2,122,Inactive Low-Value


<h4><b>Creating Segmentation Summary</b></h4>

In [40]:
segment_summary = (
    rfm
    .groupby("segments")
    .agg(
        no_of_customers=("customer_id", "count"),
        total_revenue=("monetary", "sum"),
        avg_revenue=("monetary", "mean"), median_revenue=("monetary", "median"),
        avg_recency=("recency_days", "mean"),median_recency=("recency_days", "median"),
        avg_frequency=("frequency", "mean"), median_frequency=("frequency", "median")
        
    )
    .sort_values("total_revenue", ascending=False).round(3)
)
segment_summary["revenue_share%"] = (
    segment_summary["total_revenue"]
    / segment_summary["total_revenue"].sum()
    * 100
).round(2)

segment_summary


,no_of_customers,total_revenue,avg_revenue,median_revenue,avg_recency,median_recency,avg_frequency,median_frequency,revenue_share%
segments,,,,,,,,,
Active High-Value,1316,6882440.900,5229.818,2358.885,19.452,15.0,214.218,133.0,77.23
Inactive High-Value,419,986265.391,2353.855,1516.000,125.933,100.0,73.776,57.0,11.07
Inactive Low-Value,1835,677100.082,368.992,320.620,166.538,152.0,24.519,19.0,7.60
Active Low-Value,768,365601.531,476.044,462.425,20.042,18.0,52.172,38.0,4.10


<h4><b>Exporting rfm table to SQL Database</b></h4>

In [41]:
rfm.to_sql("rfm_customers",conn,if_exists="replace",index=False)
print(pd.read_sql("SELECT COUNT(*) FROM rfm_customers",conn))
pd.read_sql("SELECT * FROM rfm_customers LIMIT 5", conn)

   COUNT(*)
0      4338


,customer_id,monetary,monetary_log,last_purchase_date,recency_days,frequency,r_score,f_score,m_score,rfm_code,segments
0,12346,77183.60,11.253955,2011-01-18 00:00:00,325,1,1,1,5,115,Inactive High-Value
1,12347,4310.00,8.368925,2011-12-07 00:00:00,2,182,5,5,5,555,Active High-Value
2,12348,1797.24,7.494564,2011-09-25 00:00:00,75,31,2,3,4,234,Inactive High-Value
3,12349,1757.55,7.472245,2011-11-21 00:00:00,18,73,4,4,4,444,Active High-Value
4,12350,334.40,5.815324,2011-02-02 00:00:00,310,17,1,2,2,122,Inactive Low-Value


### RFM Segmentation Summary

Rule-based RFM segmentation was applied to group customers into four interpretable, action-oriented segments: Active High-Value, Active Low-Value, Inactive High-Value, and Inactive Low-Value.
<br>
The results show that Active High-Value customers constitute a relatively small share of the customer base while contributing the majority of total revenue (approximately 77%), highlighting strong revenue concentration. These customers represent a critical segment for retention and loyalty-focused strategies.
